In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import random
import matplotlib.pyplot as plt
import os
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import shutil
import glob
from pathlib import Path
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model.builder_impl import SavedModelBuilder

In [3]:
def dict_product(dicts):
    result = []
    for d in dicts:
        result += list((dict(zip(d, x))) for x in product(*d.values()))
    return result

In [4]:
def samples_num_in_window(frequency, window_size_ms):
    return int(window_size_ms * frequency / 1000)

In [5]:
def emg_data_windowing(data, window_size):
    data_win = np.copy(data)
    data_x = data_win[:,:-1]
    data_y = data_win[:,-1]
    n, m = data_x.shape
    size = n * m
    residual_rows_num =  n % window_size
    if residual_rows_num != 0:
        data_x = data_x[:-residual_rows_num,:]
        data_y = data_y[:-residual_rows_num]
    data_x = data_x.reshape((-1, m * window_size))
    
    data_y = data_y.reshape((-1, window_size))
    data_y = np.array(list(map(np.mean, data_y)))
    
    mixed_classes_idxs = np.where(data_y % 1 != 0)
    
    data_win = np.c_[data_x, data_y]
    data_win = np.delete(data_win, mixed_classes_idxs, 0)
    
    return data_win

In [6]:
def read_emg(data_path):
    sessions_csv = []
    for path, _, files in os.walk(data_path):
        for name in files:
            sessions_csv.append(os.path.join(path, name))

    data = pd.concat([pd.read_csv(file, header = None) for file in sessions_csv]).values
    print('input shape', data.shape)
    
    # reshape data
    # one column - one channel
    data_x = data[:,:-1]
    data_y = data[:,-1]
    data_x = data_x.reshape((-1, 8))
    data_y = data_y.repeat(8)
    data_y = data_y.reshape((-1,1))
    data = np.concatenate((data_x, data_y), axis=1)
    print('result shape: ', data.shape)

    return data

In [7]:
from nitime.algorithms.autoregressive import AR_est_LD
from sklearn.preprocessing import StandardScaler

def autoregression_coefficients(emg, order):
    coef = AR_est_LD(emg, order=order)[0]
    return coef

In [8]:
import math

def integrated_absolute_value(segment):
    return sum([abs(s) for s in segment])

def mean_absolute_value(segment):
    return sum([abs(s) for s in segment])/len(segment)

def waveform_length(segment):
    n = len(segment)
    wl = 0
    for i in range(1, n):
        wl += abs(segment[i] - segment[i-1])
    return wl

def zero_crossing(segment):
    n = len(segment)
    zc = 0
    for i in range(n - 1):
        if segment[i] * segment[i+1] < 0:
            zc += 1
    return zc

def slope_sign_changes(segment):
    n = len(segment)
    ssc = 0
    for i in range(1, n-1):
        if segment[i-1] < segment[i] and segment[i] > segment[i+1] or segment[i-1] > segment[i] and segment[i] < segment[i+1]:
            ssc += 1
    return ssc

def root_mean_square(segment):
    return math.sqrt(sum([s*s for s in segment])/len(segment))

In [9]:
def calculate_features(data_x, channels_num, ar_features=True):
    n, m = data_x.shape
    features = []
    
    for channel in range(channels_num):
        channel_features = []
        
        # Calculate MAV, ZC, SSC, WL, RMS features
        channel_features.append(list(map(mean_absolute_value, data_x[:,channel::channels_num])))
        channel_features.append(list(map(waveform_length, data_x[:,channel::channels_num])))
        channel_features.append(list(map(zero_crossing, data_x[:,channel::channels_num])))
        channel_features.append(list(map(slope_sign_changes, data_x[:,channel::channels_num])))
        channel_features.append(list(map(root_mean_square, data_x[:,channel::channels_num])))
        
        if ar_features:
            # calculate AR6 coefficients
            ar_order = 6
            ar_coef = np.array(list(map(lambda x: autoregression_coefficients(x, ar_order), data_x[:,channel::channels_num])))
            channel_features += ar_coef.transpose().tolist()
        
        features += channel_features
    
    return np.array(features).transpose()

In [41]:
# prepare data
datasets_path = 'data/gestures-9/'
datasets = []
for dataset_name in list(os.walk(datasets_path))[0][1]:
    dataset_path = datasets_path + dataset_name
    print(dataset_path)
    session_names = list(os.walk(dataset_path))[0][1]
    sessions = []
    for session_name in session_names:
        current_session = os.path.join(dataset_path, session_name)
        print(current_session)
        sessions.append(read_emg(current_session))
    datasets.append(sessions)
    print()

data/gestures-9/p001
data/gestures-9/p001\session1
input shape (2250, 65)
result shape:  (18000, 9)
data/gestures-9/p001\session2
input shape (2250, 65)
result shape:  (18000, 9)
data/gestures-9/p001\session3
input shape (2250, 65)
result shape:  (18000, 9)

data/gestures-9/p002
data/gestures-9/p002\session1
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p002\session2
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p002\session3
input shape (2251, 65)
result shape:  (18008, 9)

data/gestures-9/p003
data/gestures-9/p003\session1
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p003\session2
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p003\session3
input shape (2251, 65)
result shape:  (18008, 9)



In [42]:
preprocessed_datasets = []
for dataset in datasets:
    preprocessed_sessions = []
    for session in dataset:
        window_samples = samples_num_in_window(200, 200)
        session_win = emg_data_windowing(session, window_samples)

        session_X = session_win[:,:-1]
        session_y = session_win[:,-1].astype('int')

        session_features_X = calculate_features(session_X, 8, True)
        # Concatenate X and y
        session_features = np.c_[session_features_X, session_y]
        preprocessed_sessions.append(session_features)
    preprocessed_datasets.append(preprocessed_sessions)

In [43]:
preprocessed_datasets[0][0].shape

(450, 89)

In [16]:
train = np.concatenate(preprocessed_datasets[0][0:2])
test = preprocessed_datasets[0][2]

Ytrain = train[:,-1].astype('int')
Xtrain = train[:,:-1]

Ytest = test[:,-1].astype('int')
Xtest = test[:,:-1]

In [17]:
NUM_READS = 1 # each row contains 1 read
NUM_SENSORS = 88 # each read has 8 sensors(channels) * 5 features

D = NUM_SENSORS * NUM_READS # number of input features
M1 = 25 # first layer number of nodes, relatively arbitrarily chosen
M2 = 15 # second hidden layer number of nodes, relatively arbitrarily chosen
M3 = 10 # third hidden layer number of nodes, relatively arbitrarily chosen

K = 9 # output layer nodes or number of classes

N = len(Ytrain)
T = np.zeros((N, K))
for i in range(N):
    T[i, Ytrain[i]] = 1 # this creates an indicator/dummy variable matrix for the output layer. We need to do this for
# two reasons. 1) it creates an NxK matrix that will be broadcastable with the predictions generated from the forward
# function and used in the cost function. 2) when we argmax the predictions, it will turn into a matrix NxK of values only
# either 1 or 0 which can directly be compared with T to test the accuracy


def initialize_weights_and_biases(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def feed_forward(W4, W3, W2, W1, b4, b3, b2, b1, X):
    Z1 = tf.matmul(X, W1)
#     Z1 = tf.nn.dropout(Z1, 0.9) #check TFLite support
    Z1 = tf.nn.relu(Z1 + b1)

    Z2 = tf.matmul(Z1, W2)
#     Z2 = tf.nn.dropout(Z2, 0.9) #check TFLite support
    Z2 = tf.nn.relu(Z2 + b2)

    Z3 = tf.nn.relu(tf.matmul(Z2, W3) + b3)
    return tf.matmul(Z3, W4) + b4, Z3

tfX = tf.placeholder(tf.float32, [None, D]) # creates placeholder variables without actually assigning values to them yet
tfY = tf.placeholder(tf.float32, [None, K]) # None means it can take any size N total number of instances


W1 = initialize_weights_and_biases([D, M1])
W2 = initialize_weights_and_biases([M1, M2])
W3 = initialize_weights_and_biases([M2, M3])
W4 = initialize_weights_and_biases([M3, K])
b1 = initialize_weights_and_biases([M1])
b2 = initialize_weights_and_biases([M2])
b3 = initialize_weights_and_biases([M3])
b4 = initialize_weights_and_biases([K])

pY_given_X, mid_layer = feed_forward(W4, W3, W2, W1, b4, b3, b2, b1, tfX)
y_max = tf.argmax(pY_given_X, dimension=1)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    labels = tfY, logits = pY_given_X))

train_model = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
predict_output = tf.argmax(pY_given_X, 1) # 1 refers to axis = 1, meaning it does argmax on each instance n

session = tf.Session()
initializer = tf.global_variables_initializer()
session.run(initializer)

for i in range(8000):
    session.run(train_model, feed_dict = {tfX: Xtrain, tfY: T})
    pred = session.run(predict_output, feed_dict = {tfX:Xtrain, tfY:T})
    if i % 250 == 0:
        print("classification_rate: {}".format(np.mean(Ytrain == pred)))

# Test Set evaluation
Ntest = len(Ytest)
Ttest = np.zeros((Ntest, K)) # test set indicator matrix
for i in range(Ntest):
    Ttest[i, Ytest[i]] = 1

predtest = session.run(predict_output, feed_dict = {tfX: Xtest, tfY: Ttest})
print("Test Set classification rate: {}".format(np.mean(Ytest == predtest)))

Instructions for updating:
Use the `axis` argument instead
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.11333333333333333
classification_rate: 0.13844444444444445
classification_rate: 0.15911111111111112
classification_rate: 0.12311111111111112
classification_rate: 0.11066666666666666
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.088
classification_rate: 0.18711111111111112
classification_rate: 0.22511111111111112
classification_rate: 0.2604444444444444
classification_rate: 0.3042222222222222
classification_rate: 0.3008888888888889
classification_rate: 0.30333333333333334
classification_rate: 0.306
classification_rate: 0.4148888888888888

In [45]:
for dataset_id in range(3):
    train = np.concatenate(preprocessed_datasets[dataset_id][0:2])
    test = preprocessed_datasets[dataset_id][2]

    Ytrain = train[:,-1].astype('int')
    Xtrain = train[:,:-1]

    Ytest = test[:,-1].astype('int')
    Xtest = test[:,:-1]
        
    results_nn = []
    for i in range(5):

        NUM_READS = 1 # each row contains 1 read
        NUM_SENSORS = 88 # each read has 8 sensors(channels) * 5 features

        D = NUM_SENSORS * NUM_READS # number of input features
        M1 = 25 # first layer number of nodes, relatively arbitrarily chosen
        M2 = 15 # second hidden layer number of nodes, relatively arbitrarily chosen
        M3 = 10 # third hidden layer number of nodes, relatively arbitrarily chosen

        K = 9 # output layer nodes or number of classes

        N = len(Ytrain)
        T = np.zeros((N, K))
        for i in range(N):
            T[i, Ytrain[i]] = 1 # this creates an indicator/dummy variable matrix for the output layer. We need to do this for
        # two reasons. 1) it creates an NxK matrix that will be broadcastable with the predictions generated from the forward
        # function and used in the cost function. 2) when we argmax the predictions, it will turn into a matrix NxK of values only
        # either 1 or 0 which can directly be compared with T to test the accuracy


        def initialize_weights_and_biases(shape):
            return tf.Variable(tf.random_normal(shape, stddev=0.01))

        def feed_forward(W4, W3, W2, W1, b4, b3, b2, b1, X):
            Z1 = tf.matmul(X, W1)
        #     Z1 = tf.nn.dropout(Z1, 0.9) #check TFLite support
            Z1 = tf.nn.relu(Z1 + b1)

            Z2 = tf.matmul(Z1, W2)
        #     Z2 = tf.nn.dropout(Z2, 0.9) #check TFLite support
            Z2 = tf.nn.relu(Z2 + b2)

            Z3 = tf.nn.relu(tf.matmul(Z2, W3) + b3)
            return tf.matmul(Z3, W4) + b4, Z3

        tfX = tf.placeholder(tf.float32, [None, D]) # creates placeholder variables without actually assigning values to them yet
        tfY = tf.placeholder(tf.float32, [None, K]) # None means it can take any size N total number of instances


        W1 = initialize_weights_and_biases([D, M1])
        W2 = initialize_weights_and_biases([M1, M2])
        W3 = initialize_weights_and_biases([M2, M3])
        W4 = initialize_weights_and_biases([M3, K])
        b1 = initialize_weights_and_biases([M1])
        b2 = initialize_weights_and_biases([M2])
        b3 = initialize_weights_and_biases([M3])
        b4 = initialize_weights_and_biases([K])

        pY_given_X, mid_layer = feed_forward(W4, W3, W2, W1, b4, b3, b2, b1, tfX)
        y_max = tf.argmax(pY_given_X, dimension=1)

        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels = tfY, logits = pY_given_X))

        train_model = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
        predict_output = tf.argmax(pY_given_X, 1) # 1 refers to axis = 1, meaning it does argmax on each instance n

        session = tf.Session()
        initializer = tf.global_variables_initializer()
        session.run(initializer)

        for i in range(8000):
            session.run(train_model, feed_dict = {tfX: Xtrain, tfY: T})
            pred = session.run(predict_output, feed_dict = {tfX:Xtrain, tfY:T})
            if i % 250 == 0:
                print("classification_rate: {}".format(np.mean(Ytrain == pred)))

        # Test Set evaluation
        Ntest = len(Ytest)
        Ttest = np.zeros((Ntest, K)) # test set indicator matrix
        for i in range(Ntest):
            Ttest[i, Ytest[i]] = 1

        predtest = session.run(predict_output, feed_dict = {tfX: Xtest, tfY: Ttest})
        acc = np.mean(Ytest == predtest)
        results_nn.append(acc)
        print("Test Set classification rate: {}".format(acc))
    print('mean: {}'.format(np.mean(results_nn)))
    print('-------------------------------------\n')

classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.27666666666666667
classification_rate: 0.4022222222222222
classification_rate: 0.7244444444444444
classification_rate: 0.8066666666666666
classification_rate: 0.8755555555555555
classification_rate: 0.8711111111111111
classification_rate: 0.9066666666666666
classification_rate: 0.9188888888888889
classification_rate: 0.8811111111111111
classification_rate: 0.7866666666666666
classification_rate: 0.9188888888888889
classification_rate: 0.9311111111111111
classification_rate: 0.7577777777777778
classification_rate: 0.7933333333333333
classification_rate: 0.8655555555555555
classification_rate: 0.8911111111111111
classification_rate: 0.9122222222222223
classification_rate: 0.9188888888888889
classification_rate: 0.9188888888888889
classification_rate: 0.9188888888888889
classification_rate: 0.9411111111111111
classification_rate: 0.9333333333333333

classification_rate: 0.7144444444444444
classification_rate: 0.64
classification_rate: 0.79
classification_rate: 0.7566666666666667
classification_rate: 0.8166666666666667
classification_rate: 0.8222222222222222
classification_rate: 0.8133333333333334
classification_rate: 0.8344444444444444
classification_rate: 0.8166666666666667
classification_rate: 0.7422222222222222
classification_rate: 0.8455555555555555
classification_rate: 0.8244444444444444
classification_rate: 0.8311111111111111
classification_rate: 0.8455555555555555
classification_rate: 0.8555555555555555
classification_rate: 0.8344444444444444
classification_rate: 0.8666666666666667
classification_rate: 0.8622222222222222
classification_rate: 0.86
classification_rate: 0.8477777777777777
classification_rate: 0.8711111111111111
classification_rate: 0.8688888888888889
Test Set classification rate: 0.6377777777777778
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.2277777777

classification_rate: 0.9422222222222222
classification_rate: 0.94
classification_rate: 0.9311111111111111
classification_rate: 0.9455555555555556
classification_rate: 0.94
classification_rate: 0.9033333333333333
classification_rate: 0.9466666666666667
classification_rate: 0.9255555555555556
classification_rate: 0.9544444444444444
classification_rate: 0.9188888888888889
classification_rate: 0.73
classification_rate: 0.9122222222222223
Test Set classification rate: 0.6733333333333333
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.1111111111111111
classification_rate: 0.2222222222222222
classification_rate: 0.42777777777777776
classification_rate: 0.42444444444444446
classification_rate: 0.5233333333333333
classification_rate: 0.8211111111111111
classification_rate: 0.8033333333333333
classification_rate: 0.8533333333333334
classification_rate: 0.8755555555555555
classification_rate: 0.8422222222222222
classification_rate: 0.90111111